In [ ]:
from openai import OpenAI
from langchain_openai import ChatOpenAI

open_ai_key = ""
MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(model=MODEL, temperature=0.3, api_key=open_ai_key)

In [ ]:
from datasets import Dataset

import json

with open('/root/ragas/data/filter_qa.json', 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

with open('/root/ragas/data/answers_rag.json', 'r', encoding='utf-8') as f:
    answers_data = json.load(f)

with open('/root/ragas/data/retrieved_contexts_rag.json', 'r', encoding='utf-8') as f:
    contexts_data = json.load(f)

data_samples = {
    'question': [],
    'answer': [],
    'contexts': [],
    'ground_truth': []
}

for qa, answer, contexts in zip(qa_data, answers_data, contexts_data):
    data_samples['question'].append(qa[0])
    data_samples['ground_truth'].append(qa[1])
    data_samples['answer'].append(answer)  
    data_samples['contexts'].append([contexts]) 


print(json.dumps(data_samples, indent=2, ensure_ascii=False))

dataset = Dataset.from_dict(data_samples)

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    context_precision,
    context_recall,
)

metrics = [
    answer_correctness,
    context_precision,
    context_recall,
]

from langchain_community.embeddings import HuggingFaceEmbeddings

EMBED_MODEL = "intfloat/e5-large-v2"

embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    model_kwargs={'device': 'cpu'}
)

results = evaluate(dataset, metrics, llm = llm, embeddings=embeddings)

results